## Import Libraries

In [ ]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.initializers import Constant
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.python.ops import array_ops
from tqdm import tqdm
from keras import backend as K
import tensorflow as tf
import keras
from math import ceil, floor
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import log_loss
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten

## Define Paths and dataframes.

In [ ]:
base_path = '/kaggle/input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/'
test_image_dir = base_path + 'stage_2_test/'
train_image_dir= base_path + 'stage_2_train/'
train_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv')
test_df = pd.read_csv('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_sample_submission.csv')


## EDA

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
def read_train(train_df):
    
    train_df['type'] = train_df.ID.str.slice(start = 13)
    train_df['filename'] = train_df.ID.str.slice(stop = 12)
    train_df = train_df.drop_duplicates()
    train_df = train_df.reset_index()
    train_df = train_df.drop('ID', axis =1)
    train_df = train_df.set_index(['filename', 'type']).unstack(level=-1)
    train_df = train_df.drop('index', axis =1)
    return train_df 

def read_test(test_df):
    test_df['type'] = test_df.ID.str.slice(start = 13)
    test_df['filename'] = test_df.ID.str.slice(stop = 12)
    test_df = test_df.drop_duplicates()
    test_df= test_df.drop('ID', axis =1)
    test_df = test_df.set_index(['filename', 'type']).unstack(level=-1)
    return test_df

In [ ]:
train_df = read_train(train_df)
test_df = read_test(test_df)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print(f'test_df shape: {test_df.shape}')
print(f'train_df shape: {train_df.shape}')

In [ ]:
df=train_df.reset_index(level=0)

In [ ]:
img_names = df['filename']

In [ ]:
del df

## Visualization

In [ ]:
def Visualize_Ten_Image(train_df , type1):
    row = 2
    col = 5
    fig = plt.figure(figsize = (15, 5)) 
    count = 0
    for i in range(len(train_df)):

        
        if train_df['Label'][type1][i] == 1:
            ds = pydicom.dcmread(train_image_dir + str(img_names[i]+'.dcm'))
            fig.add_subplot(row, col, count+1)
            plt.suptitle(type1, fontsize = 20)
            plt.imshow(ds.pixel_array, cmap=plt.cm.bone)
            plt.axis('off')
            count += 1
            if count == 10:
                break
                

In [ ]:
Visualize_Ten_Image(train_df, 'epidural')

In [ ]:
Visualize_Ten_Image(train_df, 'intraparenchymal')

In [ ]:
Visualize_Ten_Image(train_df, 'intraventricular')

In [ ]:
Visualize_Ten_Image(train_df, 'subarachnoid')

In [ ]:
Visualize_Ten_Image(train_df, 'subdural')

## Read Images and windowing

many thanks to Ryan Epp. this code below was taken from Ryan Epp. if you want to check his kernel here is the link:: [Ryan Epp](https://www.kaggle.com/reppic/gradient-sigmoid-windowing)

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    
    # Resize
    img = cv2.resize(img, SHAPE[:2], interpolation = cv2.INTER_LINEAR)
   
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)
    return bsb_img

def _read(path, SHAPE):
    dcm = pydicom.dcmread(path)
    try:
        img = bsb_window(dcm)
    except:
        img = np.zeros(SHAPE)
    return img

## Create Data Generators

In [ ]:
class Train_Val_Data_Generator(keras.utils.Sequence):
    
    def __init__(self, data, labels, batch_size = 32, img_size = (256, 256, 3), img_dir = train_image_dir, shuffle = False,*args, **kwargs):
        self.data = data
        self.ids = data.index
        self.labels = labels
        self.batch_size = batch_size
        self.img_dir = img_dir
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))
    
    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X, Y = self.__data_generate(indices)
        return X, Y

    def on_epoch_end(self):
        
        self.indices = np.arange(len(self.ids))
        
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __data_generate(self, indices):
        X = np.empty((self.batch_size, *self.img_size))
        Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
        for i, index in enumerate(indices):
            ID = self.ids[index]
            image = _read(str(self.img_dir)+str(ID)+".dcm", self.img_size)
            X[i,] = image
            Y[i,] = self.labels.iloc[index].values        
        return X, Y
    

class Test_Data_Generator(keras.utils.Sequence):
    
    def __init__(self, data, labels, batch_size = 32, img_size = (256, 256, 3), img_dir = test_image_dir, *args, **kwargs):
        self.data = data
        self.ids = data.index
        self.labels = labels
        self.img_size = img_size
        self.batch_size = batch_size
        self.img_dir = img_dir
        self.on_epoch_end()
    
    def __len__(self):
        return int(ceil(len(self.ids) / self.batch_size))
    
    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        X= self.__data_generate(indices)
        return X

    def on_epoch_end(self):
        
        self.indices = np.arange(len(self.ids))
        
    
    def __data_generate(self, indices):
            X = np.empty((self.batch_size, *self.img_size))
            
            for i, index in enumerate(indices):
                ID = self.ids[index]
                image = _read(str(self.img_dir)+str(ID)+".dcm", self.img_size)
                X[i,] = image
                return X
    
    

In [ ]:
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42).split(train_df.index)
train_idx, valid_idx = next(ss)

In [ ]:
train_gen = Train_Val_Data_Generator(train_df.iloc[train_idx], 
                                     train_df.iloc[train_idx],
                                     shuffle = False)
val_gen = Train_Val_Data_Generator(train_df.iloc[valid_idx], 
                                   train_df.iloc[valid_idx],
                                   shuffle = False)

test_gen = Test_Data_Generator(test_df, None,32, (256,256,3), test_image_dir,verbose =1)

In [ ]:
inceptionv3 =tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(256,256,3),
)

## Create the Model

In [ ]:
def build_model():
    print('Building Model..')
    model = Sequential()
    model.add(inceptionv3)
    model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(6, activation='sigmoid'))
    
    model.compile(optimizer = keras.optimizers.Adam(), 
                  loss = 'binary_crossentropy',
                  metrics =  ['acc'])
    return model

model = build_model()
    
print(f'Model Summary: {model.summary()} ')


In [ ]:
TRAIN_STEPS = int(len(train_gen) / 32)
TRAIN_STEPS

## Callbacks

In [ ]:
Early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=1, 
                                              mode='auto', baseline=None, restore_best_weights=False)

In [ ]:
model.fit_generator(generator = train_gen,
                    validation_data = val_gen,
                    steps_per_epoch = TRAIN_STEPS,
                    callbacks = [Early_stop],
                    epochs = 3)

In [ ]:
test_steps = len(test_gen)
test_steps

Many thanks to Robin Smits. The code below is taken from Robin Smits to submit Test predictions. if you want to check his kernel here is the link [Robin Smits](https://www.kaggle.com/rsmits/keras-efficientnet-b2-starter-code/notebook)

In [ ]:
def predictions(test_df, model):    
    test_preds = model.predict_generator(test_gen, verbose = 1)
    return test_preds[:test_df.iloc[range(test_df.shape[0])].shape[0]]

In [ ]:
submission_predictions =[]

In [ ]:
preds = predictions(test_df, model)
submission_predictions.append(preds)

In [ ]:
test_df.iloc[:, :] = np.average(submission_predictions, axis = 0, weights = [2**i for i in range(len(submission_predictions))])
test_df = test_df.stack().reset_index()
test_df.insert(loc = 0, column = 'ID', value = test_df['filename'].astype(str) + "_" + test_df['type'])
test_df = test_df.drop(["filename", "type"], axis=1)
test_df.to_csv('submission.csv', index = False)

print(test_df.head(12))